In [1]:
import torch
import numpy as np
import pandas as pd
from rembg import remove
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import glob
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
img_path = "images/xppen.jpg"
img = Image.open(img_path)
img = img.convert('RGB')

no_bg = remove(img)
no_bg = no_bg.convert('RGBA')
no_bg_path = f'{img_path.split(".")[0]}_no_bg.png'
no_bg.save(no_bg_path)

In [3]:
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")

use_large_model = True

if use_large_model:
    midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")
else:
    midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")

Using cache found in /home/gunjan/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /home/gunjan/.cache/torch/hub/facebookresearch_WSL-Images_main
Using cache found in /home/gunjan/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /home/gunjan/.cache/torch/hub/facebookresearch_WSL-Images_main


In [4]:
device = "cpu"
midas.to(device)

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if use_large_model:
    transform = midas_transforms.default_transform
else:
    transform = midas_transforms.small_transform

Using cache found in /home/gunjan/.cache/torch/hub/intel-isl_MiDaS_master


In [5]:
def depth(img):
    cv_image = np.array(img) 
    img = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)

    input_batch = transform(img).to(device)
    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

        output = prediction.cpu().numpy()
        formatted = (output * 255 / np.max(output)).astype('uint8')
        img = Image.fromarray(formatted)
    return img

In [6]:
depth_output = depth(img=img)
depth_output_path = f'{img_path.split(".")[0]}_depth_map.png'
depth_output.save(depth_output_path)

In [7]:
color = o3d.io.read_image(no_bg_path)
depth_img = o3d.io.read_image(depth_output_path)
rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color,
    depth_img,
    convert_rgb_to_intensity=False
)
# pinhole_intrinsics = o3d.camera.PinholeCameraIntrinsic(720, 640, 1.39072361e+03, 1.18293812e+02, 9.39469264e+02, 4.55648048e+02)
pinhole_intrinsics = o3d.camera.PinholeCameraIntrinsic(720, 640, 1.39072361e+03, 1.18293812e+02, 9.39469264e+02, 4.55648048e+02)
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, pinhole_intrinsics)

In [8]:
o3d.visualization.draw_geometries([pcd])

In [9]:
pcd.transform([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])

nb_neighbors=15 
std_ratio=1.6
# voxel_size=5e-3

cl, ind = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors,
                                                    std_ratio=std_ratio)
pcd_clean = pcd.select_by_index(ind)
pcd = pcd_clean
k = np.asarray(pcd.colors)
print(k)
print(k.shape)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
(11571876, 3)


In [105]:
import numpy as np
import open3d as o3d
from open3d.web_visualizer import draw

In [10]:
# mask = np.isclose(k[:,2],0.39215686, rtol= 1)
pcd_sel = pcd.select_by_index(np.where(np.isclose(k[:,2],0.39215686, rtol= 1)!=True)[0])
hull, _ = o3d.geometry.PointCloud.compute_convex_hull(pcd_sel)
hull.orient_triangles()
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((1, 0, 0))
o3d.visualization.draw_geometries([pcd_sel, hull_ls])    # visualize the point cloud

print(hull.get_volume())

8.745937686221349e-10


In [75]:
2.204483820572964e-11 * (35000**3)

945.1724380706584

In [90]:
# from o3d.geometry.OrientedBoundingBox import get_oriented_bounding_box
bounding_box = o3d.geometry.OrientedBoundingBox.create_from_points(pcd_sel.points, robust=True)
# create_from_points(pcd_sel.points, robust=True)

In [91]:
np.asarray(bounding_box.get_box_points())

array([[ 0.00071559, -0.00021264, -0.00082379],
       [ 0.00017643, -0.00019307, -0.00044188],
       [ 0.00074053,  0.00052054, -0.00082613],
       [ 0.00065507, -0.00021085, -0.00090932],
       [ 0.00014085,  0.00054189, -0.00052976],
       [ 0.00068001,  0.00052233, -0.00091166],
       [ 0.00011591, -0.00019129, -0.00052741],
       [ 0.00020137,  0.0005401 , -0.00044423]])

In [92]:
bounding_box.volume()

5.081432692968005e-11

In [82]:
2.2044838205729638e-11* (33000**3)

792.225350599306

In [93]:
o3d.visualization.draw_geometries([bounding_box])

In [73]:
bounding_box

OrientedBoundingBox: center: (0.00046027, 5.60573e-05, -0.000704153), extent: 0.000762067, 0.000695051, 0.000105202)

In [78]:
26/0.000762067

34117.735054791774

In [50]:
bounding_box.volume()

2.839314389368581e-11

In [94]:
bounding_box.color = [0,1,0]

In [95]:
o3d.visualization.draw_geometries([pcd_sel, bounding_box])

In [52]:
2.839314389368581e-11 * (36678.78813284009**3)

1401.0654314662424

In [ ]:
37385.232833031